## ===================================

In [1]:
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

c:\prj\team3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

c:\prj\team3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\prj\team3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
import tqdm
import os

from torchvision import datasets, models, transforms
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
# from torch.optim.adam import Adam

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time

In [5]:
transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=16, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=16, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()

학습 데이터셋 크기: 2004
테스트 데이터셋 크기: 3038
클래스: ['0', '1']


In [6]:
num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#3 vgg_models/vgg_dict{epoch}.pth')

writer.close()

#0 Loss: 0.6852 Acc: 54.9401% Time: 52.5326s
#1 Loss: 0.6738 Acc: 57.3852% Time: 100.8757s
#2 Loss: 0.6437 Acc: 62.6248% Time: 149.0655s
#3 Loss: 0.6215 Acc: 64.3713% Time: 197.2678s
#4 Loss: 0.5560 Acc: 71.1577% Time: 245.4887s
#5 Loss: 0.4669 Acc: 77.1457% Time: 293.6255s
#6 Loss: 0.4005 Acc: 81.3872% Time: 341.6439s
#7 Loss: 0.3001 Acc: 87.0758% Time: 389.8406s
#8 Loss: 0.2478 Acc: 90.0200% Time: 438.0329s
#9 Loss: 0.2356 Acc: 89.9701% Time: 488.8059s
#10 Loss: 0.1546 Acc: 93.7625% Time: 536.9804s
#11 Loss: 0.1146 Acc: 95.9082% Time: 585.2110s
#12 Loss: 0.0889 Acc: 96.7066% Time: 633.4236s
#13 Loss: 0.1146 Acc: 95.5589% Time: 681.5558s
#14 Loss: 0.0457 Acc: 98.6028% Time: 729.7275s
#15 Loss: 0.0457 Acc: 98.4531% Time: 777.9444s
#16 Loss: 0.0730 Acc: 97.8044% Time: 826.1117s
#17 Loss: 0.0268 Acc: 99.2515% Time: 874.2859s
#18 Loss: 0.0576 Acc: 97.9541% Time: 922.4243s
#19 Loss: 0.0298 Acc: 99.1018% Time: 970.5882s
#20 Loss: 0.0333 Acc: 98.8024% Time: 1018.9354s
#21 Loss: 0.0283 Acc: 9

In [6]:
import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#3 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()



# #====================================================================================================
# #4
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=16, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=16, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.00001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#4 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#4 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()


# #====================================================================================================
# #5
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#5 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#5 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================
# #6
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.0001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#6 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#6 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================
# #7
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.00001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#7 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#7 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================
# #8
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#8 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#8 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================
# #9
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.0001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#9 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#9 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================
# #10
# # import torch
# # import torch.nn as nn

# # from torchvision.models.vgg import vgg16

# torch.manual_seed(2022)
# torch.cuda.manual_seed(2022)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# model = vgg16(pretrained=True)

# # print(model)

# fc = nn.Sequential(
#     nn.Linear(25088,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,4096),
#     nn.ReLU(),
#     nn.Dropout(),
#     nn.Linear(4096,2)
# )

# model.classifier = fc
# model.to(device)

# # import tqdm
# # import os

# # from torchvision import datasets, models, transforms
# # from torchvision.datasets.cifar import CIFAR10
# # from torchvision.transforms import Compose, ToTensor, Resize
# # from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# # from torch.utils.data.dataloader import DataLoader
# # # from torch.optim.adam import Adam

# # import torch.optim as optim
# # from torch.utils.tensorboard import SummaryWriter
# # import time

# transforms_train = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# transforms_test = transforms.Compose([
#     Resize(224),
#     # RandomCrop((224,224),padding=4),
#     # RandomHorizontalFlip(p=0.5),
#     ToTensor(),
#     Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
# ])

# data_dir = './custom_dataset'
# train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
# test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
# test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

# print('학습 데이터셋 크기:', len(train_datasets))
# print('테스트 데이터셋 크기:', len(test_datasets))

# class_names = train_datasets.classes
# print('클래스:', class_names)

# lr = 0.00001
# optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# criterion = nn.CrossEntropyLoss()





# num_epochs = 30
# writer = SummaryWriter()
# start_time = time.time()

# for epoch in range(num_epochs):
#     model.train()
#     criterion.train()
#     running_loss = 0.
#     running_corrects = 0

#     for inputs, labels in train_dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         optim.zero_grad()
#         outputs = model(inputs)
#         _, preds = torch.max(outputs, 1)
#         loss = criterion(outputs, labels)
        
#         loss.backward()
#         optim.step()

#         running_loss += loss.item() * inputs.size(0)
#         running_corrects += torch.sum(preds == labels.data)

#     epoch_loss = running_loss / len(train_datasets)
#     epoch_acc = running_corrects / len(train_datasets) * 100.
#     writer.add_scalar("Loss/train", epoch_loss, epoch)
#     writer.add_scalar("Acc/train", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
#     writer.add_scalar("Loss/sum", epoch_loss, epoch)
#     writer.add_scalar("Acc/sum", epoch_acc, epoch)
#     writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

#     print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

#     ################################# 여기 ##################################
#     torch.save(model.state_dict(), f'C:/team3/vgg/models/#10 vgg_models/vgg_dict{epoch}.pth')

# writer.close()



# import os

# ################################# 여기 ##################################
# path = "C:/team3/vgg/models/#10 vgg_models"
# file_list = os.listdir(path)
# writer1 = SummaryWriter()
# start_time = time.time()
# criterion = nn.CrossEntropyLoss()

# for epoch, file in enumerate(file_list):
#     dict_model = torch.load(path + "/" + file, map_location=device)
#     model.load_state_dict(dict_model)
#     model.eval()
#     model.to("cuda")

#     with torch.no_grad():
#         test_loss = 0.
#         test_corrects = 0

#         for inputs, labels in test_dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)
#             loss = criterion(outputs, labels)

#             test_loss += loss.item() * inputs.size(0)
#             test_corrects += torch.sum(preds == labels.data)

#     epoch_loss1 = test_loss / len(test_datasets)
#     epoch_acc1 = test_corrects / len(test_datasets) * 100.
#     print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

#     writer1.add_scalar("Loss/test", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/test", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
#     writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
#     writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
#     writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

# writer1.close()

# #====================================================================================================


# #====================================================================================================


# #====================================================================================================


# #====================================================================================================


# #====================================================================================================


# #====================================================================================================

#0 [Test Phase] Loss: 0.4971 Acc: 94.7663% Time: 31.9919s
#1 [Test Phase] Loss: 0.6188 Acc: 69.8815% Time: 61.1377s
#2 [Test Phase] Loss: 0.9329 Acc: 70.7373% Time: 90.3333s
#3 [Test Phase] Loss: 1.0726 Acc: 71.0994% Time: 119.3640s
#4 [Test Phase] Loss: 2.4864 Acc: 51.5800% Time: 148.4936s
#5 [Test Phase] Loss: 1.8906 Acc: 60.8624% Time: 177.5017s
#6 [Test Phase] Loss: 1.6509 Acc: 66.1290% Time: 206.5593s
#7 [Test Phase] Loss: 1.8331 Acc: 61.8828% Time: 235.7056s
#8 [Test Phase] Loss: 0.8895 Acc: 76.0369% Time: 264.9047s
#9 [Test Phase] Loss: 1.4885 Acc: 73.2390% Time: 293.9499s
#10 [Test Phase] Loss: 1.5791 Acc: 66.1290% Time: 322.9503s
#11 [Test Phase] Loss: 1.4289 Acc: 70.8032% Time: 352.1160s
#12 [Test Phase] Loss: 0.8828 Acc: 32.7847% Time: 381.2662s
#13 [Test Phase] Loss: 1.1329 Acc: 72.3173% Time: 410.2940s
#14 [Test Phase] Loss: 1.3758 Acc: 73.0086% Time: 439.4808s
#15 [Test Phase] Loss: 1.6489 Acc: 74.0290% Time: 468.6494s
#16 [Test Phase] Loss: 1.7201 Acc: 73.6011% Time: 497

c:\prj\team3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\prj\team3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


학습 데이터셋 크기: 2004
테스트 데이터셋 크기: 3038
클래스: ['0', '1']


AttributeError: 'SGD' object has no attribute 'SGD'

## ===================================

In [7]:
#4
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

import tqdm
import os

from torchvision import datasets, models, transforms
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
# from torch.optim.adam import Adam

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=16, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=16, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.00001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#4 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#4 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

학습 데이터셋 크기: 2004
테스트 데이터셋 크기: 3038
클래스: ['0', '1']
#0 Loss: 0.6928 Acc: 50.8483% Time: 48.2255s
#1 Loss: 0.6942 Acc: 49.4511% Time: 96.3623s
#2 Loss: 0.6941 Acc: 50.2495% Time: 144.6127s
#3 Loss: 0.6925 Acc: 52.5948% Time: 192.9967s
#4 Loss: 0.6901 Acc: 52.6946% Time: 241.2597s
#5 Loss: 0.6888 Acc: 54.1916% Time: 289.5004s
#6 Loss: 0.6901 Acc: 52.8443% Time: 337.6039s
#7 Loss: 0.6871 Acc: 55.6387% Time: 385.8393s
#8 Loss: 0.6898 Acc: 53.9920% Time: 433.9560s
#9 Loss: 0.6889 Acc: 54.2914% Time: 482.1273s
#10 Loss: 0.6878 Acc: 54.9900% Time: 530.3976s
#11 Loss: 0.6868 Acc: 55.5389% Time: 578.6705s
#12 Loss: 0.6881 Acc: 55.1397% Time: 626.8979s
#13 Loss: 0.6846 Acc: 55.7385% Time: 675.0507s
#14 Loss: 0.6841 Acc: 57.3852% Time: 723.3037s
#15 Loss: 0.6849 Acc: 56.5868% Time: 771.4250s
#16 Loss: 0.6837 Acc: 57.1357% Time: 819.5993s
#17 Loss: 0.6813 Acc: 58.3832% Time: 867.8482s
#18 Loss: 0.6834 Acc: 56.8862% Time: 916.0237s
#19 Loss: 0.6826 Acc: 57.6347% Time: 964.2092s
#20 Loss: 0.6818 Acc:

In [ ]:
#5
import torch
import torch.nn as nn

from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

import tqdm
import os

from torchvision import datasets, models, transforms
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import Compose, ToTensor, Resize
from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
from torch.utils.data.dataloader import DataLoader
# from torch.optim.adam import Adam

import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#5 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#5 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 

In [ ]:
#6
# import torch
# import torch.nn as nn

# from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

# import tqdm
# import os

# from torchvision import datasets, models, transforms
# from torchvision.datasets.cifar import CIFAR10
# from torchvision.transforms import Compose, ToTensor, Resize
# from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# from torch.utils.data.dataloader import DataLoader
# # from torch.optim.adam import Adam

# import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
# import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.0001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#6 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#6 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 

In [ ]:
#7
# import torch
# import torch.nn as nn

# from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

# import tqdm
# import os

# from torchvision import datasets, models, transforms
# from torchvision.datasets.cifar import CIFAR10
# from torchvision.transforms import Compose, ToTensor, Resize
# from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# from torch.utils.data.dataloader import DataLoader
# # from torch.optim.adam import Adam

# import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
# import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.00001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#7 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#7 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 

In [ ]:
#8
# import torch
# import torch.nn as nn

# from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

# import tqdm
# import os

# from torchvision import datasets, models, transforms
# from torchvision.datasets.cifar import CIFAR10
# from torchvision.transforms import Compose, ToTensor, Resize
# from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# from torch.utils.data.dataloader import DataLoader
# # from torch.optim.adam import Adam

# import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
# import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#8 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#8 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 

In [ ]:
#9
# import torch
# import torch.nn as nn

# from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

# import tqdm
# import os

# from torchvision import datasets, models, transforms
# from torchvision.datasets.cifar import CIFAR10
# from torchvision.transforms import Compose, ToTensor, Resize
# from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# from torch.utils.data.dataloader import DataLoader
# # from torch.optim.adam import Adam

# import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
# import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.0001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#9 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#9 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 

In [ ]:
#10
# import torch
# import torch.nn as nn

# from torchvision.models.vgg import vgg16

torch.manual_seed(2022)
torch.cuda.manual_seed(2022)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = vgg16(pretrained=True)

# print(model)

fc = nn.Sequential(
    nn.Linear(25088,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096,2)
)

model.classifier = fc
model.to(device)

# import tqdm
# import os

# from torchvision import datasets, models, transforms
# from torchvision.datasets.cifar import CIFAR10
# from torchvision.transforms import Compose, ToTensor, Resize
# from torchvision.transforms import RandomHorizontalFlip, RandomCrop, Normalize
# from torch.utils.data.dataloader import DataLoader
# # from torch.optim.adam import Adam

# import torch.optim as optim
# from torch.utils.tensorboard import SummaryWriter
# import time

transforms_train = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

transforms_test = transforms.Compose([
    Resize(224),
    # RandomCrop((224,224),padding=4),
    # RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914,0.4822,0.4465) , std = (0.247,0.243,0.261))
])

data_dir = './custom_dataset'
train_datasets = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_datasets = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=8, shuffle=True, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=8, shuffle=True, num_workers=4)

print('학습 데이터셋 크기:', len(train_datasets))
print('테스트 데이터셋 크기:', len(test_datasets))

class_names = train_datasets.classes
print('클래스:', class_names)

lr = 0.00001
optim = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()





num_epochs = 30
writer = SummaryWriter()
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    criterion.train()
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optim.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optim.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.
    writer.add_scalar("Loss/train", epoch_loss, epoch)
    writer.add_scalar("Acc/train", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/train", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)
    writer.add_scalar("Loss/sum", epoch_loss, epoch)
    writer.add_scalar("Acc/sum", epoch_acc, epoch)
    writer.add_scalars("Loss_Acc/sum", {'epoch_loss_train': epoch_loss, 'epoch_acc_train': epoch_acc/100}, epoch)

    print('#{} Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

    ################################# 여기 ##################################
    torch.save(model.state_dict(), f'C:/team3/vgg/models/#10 vgg_models/vgg_dict{epoch}.pth')

writer.close()



import os

################################# 여기 ##################################
path = "C:/team3/vgg/models/#10 vgg_models"
file_list = os.listdir(path)
writer1 = SummaryWriter()
start_time = time.time()
criterion = nn.CrossEntropyLoss()

for epoch, file in enumerate(file_list):
    dict_model = torch.load(path + "/" + file, map_location=device)
    model.load_state_dict(dict_model)
    model.eval()
    model.to("cuda")

    with torch.no_grad():
        test_loss = 0.
        test_corrects = 0

        for inputs, labels in test_dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)

    epoch_loss1 = test_loss / len(test_datasets)
    epoch_acc1 = test_corrects / len(test_datasets) * 100.
    print('#{} [Test Phase] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss1, epoch_acc1, time.time() - start_time))

    writer1.add_scalar("Loss/test", epoch_loss1, epoch)
    writer1.add_scalar("Acc/test", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/test", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)
    writer1.add_scalar("Loss/sum", epoch_loss1, epoch)
    writer1.add_scalar("Acc/sum", epoch_acc1, epoch)
    writer1.add_scalars("Loss_Acc/sum", {'epoch_loss_test': epoch_loss1, 'epoch_acc_test': epoch_acc1/100}, epoch)

writer1.close()

: 